In [1]:
!pip install Django

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 1.1 MB/s eta 0:00:0000:0100:010m
  Using cached backports.zoneinfo-0.2.1-cp38-cp38-macosx_10_14_x86_64.whl (35 kB)
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
  Using cached asgiref-3.6.0-py3-none-any.whl (23 kB)


In [2]:
import django

print(django.get_version())

4.1.5


In [1]:
!pip install django_plotly_dash

  Using cached django_plotly_dash-2.1.3-py3-none-any.whl (55 kB)
  Using cached channels-4.0.0-py3-none-any.whl (28 kB)
  Using cached Django-3.2.16-py3-none-any.whl (7.9 MB)
  Using cached dpd_components-0.1.0.tar.gz (4.3 kB)
  Preparing metadata (setup.py) ... done
  Using cached plotly-5.11.0-py2.py3-none-any.whl (15.3 MB)
  Using cached dash-2.7.1-py3-none-any.whl (9.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 177.8 kB/s eta 0:00:00a 0:00:01
  Using cached Flask-2.2.2-py3-none-any.whl (101 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached tenacity-8.1.0-py3-none-any.whl (23 kB)
  Created wheel for dpd-components: filename=dpd_components-0.1.0-py3-none-any.whl size=4535 sha256=43ea7fcd6b180006d

In [2]:
!pip install dpd_static_support

  Using cached dpd_static_support-0.0.5-py3-none-any.whl (509 kB)


In [3]:
!pip install channels_redis

  Using cached channels_redis-4.0.0-py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.6/237.6 kB 251.9 kB/s eta 0:00:00a 0:00:01
  Using cached msgpack-1.0.4-cp38-cp38-macosx_10_9_x86_64.whl (73 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)


In [4]:
!pip install dash_renderer

  Using cached dash_renderer-1.9.1.tar.gz (1.0 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for dash_renderer: filename=dash_renderer-1.9.1-py3-none-any.whl size=1014850 sha256=bdcf170175bc216a21dc84b80c9f067e8c55db7e8b25841b99f573453f337e25
  Stored in directory: /Users/kritkorns/Library/Caches/pip/wheels/ac/7e/fd/807844722d79d8babcd27b16e5f7ecc7b476c45ca607c11729
Successfully built dash_renderer
